In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import cv2
from typing import Tuple, List
import os
import sys
from tqdm import tqdm

In [2]:
print(f'Tensorflow version: {tf.__version__}')
print(f'Pandas version: {pd.__version__}')
print(f'NumPy version: {np.__version__}')
print(f'OpenCV version: {cv2.__version__}')
!python --version

Tensorflow version: 2.3.1
Pandas version: 1.1.1
NumPy version: 1.18.5
OpenCV version: 4.4.0


Python 3.6.10 :: Anaconda, Inc.


In [3]:
"""
We turn this on to prevent tensorflow from throwing a fit about
things that take longer than the batch training in the model.fit
call (namely, the tensorboard callback can take more time to
execute than the batch training iteration itself).

NOTE: This can be disabled simply by commenting it out. If you
think there is a weird tensorflow issue happening, do that to see
the full tensorflow logs during runtime.
"""
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## Utility Functions

In [4]:
def getDirsAndClasses(root: str, file: str) -> Tuple[List[str], List[int]]:
    imageDirs = []
    classes = []
    line = ""
    with open(root + file, "r") as f:
        for line in tqdm(f):
            imageDir, clazz = line.split()
            imageDirs.append(imageDir)
            classes.append(int(clazz))
    return imageDirs, classes

In [5]:
def preprocessImage(img: np.ndarray) -> tf.Tensor:
    return img.astype(np.float16) / 255.0

In [6]:
def makeOneHot(value: int, size: int) -> np.ndarray:
    onehot = np.zeros(size)
    onehot[value] = 1
    return onehot

In [7]:
def balanceData(data: pd.DataFrame) -> pd.DataFrame:
    data = data.groupby("class")
    data = data.apply(lambda x: x.sample(data.size().min()).reset_index(drop=True))
    return data.reset_index(drop=True)

In [8]:
def makeModel(inputShape: Tuple[int]) -> keras.Model:
    """
    Source: https://www.tensorflow.org/guide/keras/functional#a_toy_resnet_model
    
    Note that I tend to prefer the super-explicit (if somewhat verbose) style. 
    This style is technically unnecessary, but it helps with readability.
    """
    inputs = keras.Input(shape=inputShape, name="Input")
    x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(inputs)
    x = layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu")(x)
    block_1_output = layers.MaxPooling2D(pool_size=(3, 3), strides=(3, 3))(x)

    x = layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding="same")(block_1_output)
    x = layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding="same")(x)
    block_2_output = layers.add([x, block_1_output])

    x = layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding="same")(block_2_output)
    x = layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding="same")(x)
    block_3_output = layers.add([x, block_2_output])

    x = layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu")(block_3_output)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(units=256, activation="relu")(x)
    x = layers.Dense(units=256, activation="relu")(x)
    x = layers.Dense(units=256, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units=2, activation="softmax")(x)

    return keras.Model(inputs=inputs, outputs=outputs, name="Simple_ResNet")

## Global Variables

In [9]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="accuracy",
        min_delta=0.01,
        patience=3,
        verbose=1
    ),
    tf.keras.callbacks.TensorBoard(
        log_dir="logs",
        write_graph=True,
        write_images=True
    )
]

root = os.getcwd() + "\\Data\\CNR-EXT-150x150"
# train_imageDirs, train_classes = getDirsAndClasses(root, "\\LABELS\\train.txt")
# test_imageDirs, test_classes = getDirsAndClasses(root, "\\LABELS\\test.txt")
# val_imageDirs, val_classes = getDirsAndClasses(root, "\\LABELS\\val.txt")
imageDirs, classes = getDirsAndClasses(root, "\\LABELS\\all.txt")

classDict = {
    0: "free",
    1: "busy"
}

batchSize = 128

144965it [00:00, 872530.55it/s]


## Data Acquisition and Preprocessing

In [10]:
data = pd.DataFrame([
            {
                "image": preprocessImage(cv2.imread(root + "\\PATCHES\\" + filename)),
                "class": clazz,
                "weather": filename[0]
            }
            for filename, clazz in tqdm(zip(imageDirs, classes))
        ])

144965it [03:08, 770.03it/s]


In [11]:
data.groupby("class")["class"].value_counts()

class  class
0      0        65658
1      1        79307
Name: class, dtype: int64

In [12]:
data.groupby("weather")["weather"].value_counts()

weather  weather
O        O          44243
R        R          37544
S        S          63178
Name: weather, dtype: int64

In [13]:
data.groupby(["class", "weather"])["class"].value_counts()

class  weather  class
0      O        0        21067
       R        0        18926
       S        0        25665
1      O        1        23176
       R        1        18618
       S        1        37513
Name: class, dtype: int64

In [14]:
classes = list(data.groupby("class").groups.keys())

In [15]:
data["onehot"] = data["class"].apply(
    func=lambda x: makeOneHot(classes.index(x), len(classes))
)

### Split the data into train and test subsets

In [16]:
train = data.groupby("class").sample(frac=0.8)
train.groupby("class")["class"].value_counts()

class  class
0      0        52526
1      1        63446
Name: class, dtype: int64

In [17]:
test = data.drop(train.index).reset_index(drop=True)
test.groupby("class")["class"].value_counts()

class  class
0      0        13132
1      1        15861
Name: class, dtype: int64

In [18]:
train = train.reset_index(drop=True)

In [19]:
train = tf.data.Dataset.from_tensor_slices(
    (
        np.array(train["image"].values.tolist()),
        np.array(train["onehot"].values.tolist())
    )
).shuffle(
    buffer_size=len(train),
    reshuffle_each_iteration=True
).batch(batchSize)

In [20]:
test = tf.data.Dataset.from_tensor_slices(
    (
        np.array(test["image"].values.tolist()),
        np.array(test["onehot"].values.tolist())
    )
).shuffle(
    buffer_size=len(test),
    reshuffle_each_iteration=True
).batch(batchSize)

## Model Definition

In [21]:
model = makeModel(inputShape=data.loc[0, "image"].shape)

In [22]:
model.compile(
    optimizer="adam",
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [23]:
model.summary()

Model: "Simple_ResNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 148, 148, 32) 896         Input[0][0]                      
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 146, 146, 64) 18496       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 48, 48, 64)   0           conv2d_1[0][0]                   
______________________________________________________________________________________

## Training and Evaluation

In [24]:
model.fit(
    train,
    epochs=100,
    callbacks=callbacks
)

Epoch 1/100
907/907 [==============================] - 172s 190ms/step - loss: 0.4173 - accuracy: 0.8915
Epoch 2/100
907/907 [==============================] - 178s 197ms/step - loss: 0.3693 - accuracy: 0.9416
Epoch 3/100
907/907 [==============================] - 182s 201ms/step - loss: 0.3573 - accuracy: 0.9544
Epoch 4/100
907/907 [==============================] - 188s 208ms/step - loss: 0.3452 - accuracy: 0.9669
Epoch 5/100
907/907 [==============================] - 191s 211ms/step - loss: 0.3387 - accuracy: 0.9739
Epoch 6/100
907/907 [==============================] - 196s 216ms/step - loss: 0.3356 - accuracy: 0.9767
Epoch 7/100
907/907 [==============================] - 201s 221ms/step - loss: 0.3350 - accuracy: 0.9776
Epoch 8/100
907/907 [==============================] - 201s 221ms/step - loss: 0.3332 - accuracy: 0.9794
Epoch 9/100
907/907 [==============================] - 202s 223ms/step - loss: 0.3345 - accuracy: 0.9781
Epoch 10/100
907/907 [==============================] -

In [25]:
loss, accuracy = model.evaluate(test)

227/227 [==============================] - 18s 81ms/step - loss: 0.3333 - accuracy: 0.9783


In [27]:
model.predict(test)

array([[1.3537792e-03, 9.9864620e-01],
       [9.7214478e-01, 2.7855201e-02],
       [1.1007137e-07, 9.9999988e-01],
       ...,
       [0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [1.9484680e-34, 1.0000000e+00]], dtype=float32)

## Results

In [28]:
print(f'Test Loss: {loss}\nTest Accuracy: {accuracy}')

Test Loss: 0.3332807123661041
Test Accuracy: 0.9783051013946533


In [29]:
model.save("Models/SimpleResNet")